In [10]:
#Logistic Regression with stopwords.
import os
import math
import re
import sys
import string

W_vector = {}
W_new = {}
mu = 0.01
vocabulary = set()

def get_data(X, Y, file_location, spam):  
    files = os.listdir(file_location)
    for file in files:
        f = open(os.path.join(file_location, file), 'r', encoding = "ISO-8859-1")
        text = f.read()
        text = re.sub(r'(?:^| )\w(?:$| )', ' ', text).strip()
        translator = str.maketrans("", "", string.punctuation)
        translation = str.maketrans("", "", string.digits)
        answer = text.translate(translator)
        text = answer.strip().lower()
        
        vector = {}
        vector["0"] = 1.0
        for word in text.split():
            vocabulary.add(word)
            if vector.get(word) is None:
                vector[word] = 0
            vector[word] += 1
        
        X.append(vector)
        if spam:
            Y.append(0)
        else:
            Y.append(1)
    return X, Y
            
def conditional_probability(con_x):
    global W_vector
    WX = W_vector["0"]
    for word in con_x:
        if word in W_vector:
            WX += W_vector[word] * con_x[word]

    if WX >= 700:
        return 1
    
    exp_WX = math.exp(WX)
    prob = exp_WX / (1 + exp_WX)
    return prob

def fit_model(X, Y):
    global W_vector, W_new
    W_new = W_vector.copy()
    for i in range(no_iteration):
        for word in W_vector:
            sum = 0
            for j in range(len(Y)):
                yl = Y[j]
                con_x = X[j]
                if word in con_x:
                    prob = conditional_probability(con_x)
                    sum += con_x[word] * (yl - prob)
            W_new[word] = W_vector[word] + mu * sum - lamda * mu * W_vector[word]
        W_vector = W_new.copy()

def predict_model(file_location, spam):
    total = 0
    correct_class = 0
    files = os.listdir(file_location)
    for file in files:
        total += 1
        f = open(os.path.join(file_location, file), 'r', encoding = "ISO-8859-1")
        text = f.read()
        text = re.sub(r'(?:^| )\w(?:$| )', ' ', text).strip()
        translator = str.maketrans("", "", string.punctuation)
        translation = str.maketrans("", "", string.digits)
        answer = text.translate(translator)
        text = answer.strip().lower()
        
        vector = {}
        for word in text.split():
            if word in W_vector:
                if vector.get(word) is None:
                    vector[word] = 0
                vector[word] += 1
        
        WX = W_vector["0"]
        for word in vector:
            WX += W_vector[word] * vector[word]
        
        if spam:
            if WX <= 0:
                correct_class += 1
        else:
            if WX > 0:
                correct_class += 1
    
    return correct_class, total

if __name__ == '__main__':
    X = []
    Y = []
    lamda = .001
    no_iteration = int(input("Enter number of iterations: "))
    no_iterations = no_iteration
        train_spam_location = 'assignment3_train/train/spam'
    train_ham_location = 'assignment3_train/train/ham'
    
    X, Y = get_data(X, Y, train_spam_location, True)
    X, Y = get_data(X, Y,train_ham_location, False)

    W_vector["0"] = 1
    for word in vocabulary:
        W_vector[word] = 1
    
    fit_model(X, Y)
    
    test_spam_location = 'assignment3_test/test/spam'
    test_ham_location = 'assignment3_test/test/ham'
    
    spam_correct, spam_total = predict_model(test_spam_location, True)
    ham_correct, ham_total = predict_model(test_ham_location, False)
    accuracy = (spam_correct + ham_correct) / (spam_total + ham_total) 
    print ("Accuracy:", accuracy)

Enter number of iterations: 500
Accuracy: 0.893305439330544
